<a href="https://colab.research.google.com/github/Souravdey15/Google-news-Webscraper/blob/master/summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import datetime
import spacy
spacy.load('en_core_web_sm')
import heapq
import json
import os.path
import pickle
import random
import re
import string
from collections import Counter
from pprint import pprint

import matplotlib
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd

import tensorflow as tf
from nltk import ne_chunk
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from spacy import displacy
from wordcloud import WordCloud
import os
from os import path

In [0]:
if(path.exists("summaries")!=True):
    os.mkdir("summaries")

stop_words = set(
        stopwords.words('english') +
        ['i', 'he', 'me', 'she', 'it', 'them', 'her', 'him'])

In [0]:
def remove_punc(sent):
    #
    punctuations = '''!()-[]{};'"\,<>/?@#%^&*_~'''
    for x in sent:
        if x in punctuations:
            sent = sent.replace(x, " ")
    return sent


In [0]:

def preprocess(sent):
    #
    sent = remove_punc(sent)
    sent = nltk.word_tokenize(sent, language='english')
    lemmatizer = WordNetLemmatizer()
    sent = [lemmatizer.lemmatize(x) for x in sent]
    sent = ' '.join(sent)
    filtered_sentence = [
        w for w in sent.split(' ') if not w.lower() in stop_words
    ]

    return ' '.join(filtered_sentence).lower()


In [0]:
def weighted_freq(sent):
    word_frequencies = {}
    for word in sent:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / maximum_frequncy)

    return word_frequencies

In [0]:
def sent_score_calc(text, word_frequencies):
    sentence_list = nltk.sent_tokenize(text)
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
    return sentence_scores


In [0]:
def extractive_summary(f, docu):
    max_freq = weighted_freq(docu)
    sent_scores = sent_score_calc(f, max_freq)
    no_of_lines = len(docu.split('.'))
    summary_sentences = heapq.nlargest(int(no_of_lines / 3),
                                       sent_scores,
                                       key=sent_scores.get)
    #summary_sentences =sorted(sent_scores, key=sent_scores.get, reverse=True)[:int(no_of_lines/2)]  SUMMARY OF EACH LINE CAN BE GENERATED

    summary = ' '.join(i.capitalize() for i in summary_sentences)
    print(summary)
    fileName = docu
    try:
        with open("summaries/"+fileName, mode='x') as f:
            f.write(summary)
    except:
        print(f+" already created")
        pass
    return summary

In [0]:

def return_context(docu):

    nlp = spacy.load('en_core_web_sm')
    doc = nlp(docu)
    fin_dic = {}
    for ent in doc.ents:
        fin_dic[ent.text] = ent.label_
    return json.dumps(fin_dic, sort_keys=True)

In [0]:
for fileName in os.listdir("transcriptions/"):

    f = open(fileName, 'r').read().lower()

    f = re.sub(r'\s+', ' ', f)
    no_of_lines = len(open('convotext.txt', 'r').readlines())
    

    " ".join(f.split("\n"))

    extractive_summary(" ".join(f.split("\n")), docu=f)